In [1]:
!date

Thu Mar  4 23:33:26 IST 2021


In [2]:
import pandas as pd
import numpy as np

In [3]:
base_path="../eit_basic_dual_unreduced_experiments/"

In [4]:
!ls $base_path

exp_1      exp_1_npca exp_2      exp_2b     exp_2e
exp_1_nmf  exp_1basic exp_2a     exp_2d     exp_3


### Hypothesis 1:
 * 1.1 Reduced data provides better (return/risk) for lower values of k

In [5]:
df_dict={}
required_dirs=["exp_1","exp_1_npca","exp_1_nmf"]
for req_dir in required_dirs:
    for key0 in ["dual","basic"]:
        df=pd.read_csv(base_path+req_dir+f"/{key0}_results_exp_1_transposed.csv")
        key1=req_dir.split("_")[-1]
        key=(key0,key1)
        df_dict[key]=df

In [6]:
[key for key in df_dict.keys()]

[('dual', '1'),
 ('basic', '1'),
 ('dual', 'npca'),
 ('basic', 'npca'),
 ('dual', 'nmf'),
 ('basic', 'nmf')]

In [7]:
print (list(map(lambda x: len(x.columns),df_dict.values())))

[35, 32, 35, 32, 35, 32]


In [8]:
def add_return_pu_risk(df):
    #For simple
    df["return_pu_risk_sample"]=(df["in_excess_return"])/(df["in_tr_err"].abs())
    df["return_pu_risk_oot"]=(df["out_excess_return"])/(df["out_tr_err"].abs())
    #For transposed
    df["return_pu_risk_sample_transpose"]=(df["in_excess_return_transpose"])/(df["in_tr_err_transpose"].abs())
    df["return_pu_risk_oot_transpose"]=(df["out_excess_return_transpose"])/(df["out_tr_err_transpose"].abs())
    #Risk
    #df["return_pu_risk_oot"]=df["out_excess_return"]/df["out_tr_err"].abs()
    df["k/xii"]=df["k"]/(df["xii"]*df["xii"])
    return (df)

In [9]:
for key,val in df_dict.items():
    df_dict.update({key:add_return_pu_risk(val)})

In [10]:
print (list(map(lambda x: len(x.columns),df_dict.values())))

[40, 37, 40, 37, 40, 37]


In [11]:
df_dict[('dual', '1')].columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'output', 'T', 'C', 'file', 'm', 'lbuck',
       'pho', 'lamda', 'nuh', 'xii', 'k', 'p', 'f', 'w_risk', 'w_return',
       'w_risk_down', 'index', 'bucket', 'kernel_size', 'problem_status',
       'z_value', 'in_excess_return', 'in_tr_err', 'out_excess_return',
       'out_tr_err', 'portfolio_size', 'z_linear', 'chk',
       'in_excess_return_transpose', 'in_tr_err_transpose',
       'out_excess_return_transpose', 'out_tr_err_transpose',
       'portfolio_size_transpose', 'return_pu_risk_sample',
       'return_pu_risk_oot', 'return_pu_risk_sample_transpose',
       'return_pu_risk_oot_transpose', 'k/xii'],
      dtype='object')

In [26]:
pd.options.display.float_format="{:e}".format

### Effect of k

In [27]:
z_slopes=pd.DataFrame()
for key,df_val in df_dict.items():
    z_step=df_dict[key].groupby(["k"])[["return_pu_risk_sample_transpose","return_pu_risk_oot_transpose"]].mean()
    slopes = z_step.apply(lambda x: np.polyfit(z_step.index, x, 1)[0])
    z_slopes[key]=slopes
    z_slopes.columns = pd.MultiIndex.from_tuples(z_slopes.columns, names=['EIT_type','dataset'])
print (z_step.index)
display(z_slopes[["dual","basic"]])

Int64Index([12, 16, 25], dtype='int64', name='k')


EIT_type                                 dual                             \
dataset                                     1         npca           nmf   
return_pu_risk_sample_transpose  1.356284e-07 1.961903e-06 -7.874494e-07   
return_pu_risk_oot_transpose    -4.498208e-07 1.359339e-06  2.753823e-06   

EIT_type                               basic                             
dataset                                    1         npca           nmf  
return_pu_risk_sample_transpose 1.962252e-05 3.507791e-05  6.306736e-07  
return_pu_risk_oot_transpose    1.032645e-04 2.727054e-05 -3.150138e-05

In [14]:
z_slopes.to_latex("prop_2a.tex",float_format="%.5f",escape=True,multicolumn=True,\
                 caption="Slope of risk/return vs k")

### Effect of pho

In [15]:
z_slopes=pd.DataFrame()
for key,df_val in df_dict.items():
    z_step=df_dict[key].groupby(["pho"])[["return_pu_risk_sample","return_pu_risk_oot"]].mean()
    slopes = z_step.apply(lambda x: np.polyfit(z_step.index, x, 1)[0])
    z_slopes[key]=slopes
    z_slopes.columns = pd.MultiIndex.from_tuples(z_slopes.columns, names=['EIT_type','dataset'])
print (z_step.index)
display(z_slopes[["dual","basic"]])

Float64Index([0.2, 0.3, 0.4], dtype='float64', name='pho')


EIT_type                       dual                                     basic  \
dataset                           1          npca           nmf             1   
return_pu_risk_sample  5.535120e-03 -2.942339e-01  4.733874e-02  1.496724e-03   
return_pu_risk_oot    -6.281369e-03 -1.411890e-01 -6.572610e-02 -5.886736e-04   

EIT_type                                           
dataset                        npca           nmf  
return_pu_risk_sample -4.325654e-02  7.698764e-03  
return_pu_risk_oot     9.324724e-02 -9.357021e-03

In [ ]:
-6.281369e-03